In [15]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
import pandas as pd

# Load the dataset
file_path = '/home/williamfishbu_snhu/Desktop/aac_shelter_outcomes.csv'
df = pd.read_csv(file_path)

# Initialize the Dash app
app = JupyterDash(__name__)

# Grazioso Salvare’s logo
image_filename = '/home/williamfishbu_snhu/Desktop/Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# App layout
app.layout = html.Div([
    html.Center(html.B(html.H1('William Fishburn/CS-340 Dashboard'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Hr(),
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset'
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto',
        }
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# Define callback to update data table based on filter
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'Water':
        filtered_df = df[(df['breed'].str.contains('Labrador Retriever Mix|Chesapeake Bay Retriever|Newfoundland')) &
                         (df['sex_upon_outcome'] == 'Intact Female') &
                         (df['age_upon_outcome_in_weeks'] >= 26) &
                         (df['age_upon_outcome_in_weeks'] <= 156)]
    elif filter_type == 'Mountain':
        filtered_df = df[(df['breed'].str.contains('German Shepherd|Alaskan Malamute|Old English Sheepdog|Siberian Husky|Rottweiler')) &
                         (df['sex_upon_outcome'] == 'Intact Male') &
                         (df['age_upon_outcome_in_weeks'] >= 26) &
                         (df['age_upon_outcome_in_weeks'] <= 156)]
    elif filter_type == 'Disaster':
        filtered_df = df[(df['breed'].str.contains('Doberman Pinscher|German Shepherd|Golden Retriever|Bloodhound|Rottweiler'))]
    else:
        filtered_df = df

    return filtered_df.to_dict('records')

# Define callback to update pie chart based on data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame(viewData)
    if not dff.empty:
        fig = px.pie(dff, names='breed', title='Preferred Animals')
        return [dcc.Graph(figure=fig)]
    return []

# Define callback to update map based on data table
# Callback to update map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)


def update_map(viewData, index):
    if viewData is None or len(viewData) == 0:
        return [html.Div("No data available")]

    dff = pd.DataFrame.from_dict(viewData)
    
    # Check if index is None or empty
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
    
    # Ensure the index is within bounds
    if row >= len(dff):
        row = 0
    
    # Handle missing or invalid location data
    try:
        lat = dff.iloc[row]['location_lat']
        lon = dff.iloc[row]['location_long']
        if pd.isnull(lat) or pd.isnull(lon):
            lat, lon = 30.75, -97.48  # Default to Austin, TX
    except (KeyError, IndexError):
        lat, lon = 30.75, -97.48  # Default to Austin, TX

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon],
                      children=[
                          dl.Tooltip(dff.iloc[row]['breed']),
                          dl.Popup([
                              html.H1("Animal Name"),
                              html.P(dff.iloc[row]['name'])
                          ])
                      ])
        ])
    ]

# Run the Dash app
app.run_server(debug=True)


Dash app running on http://127.0.0.1:10231/
